In [1]:
from src.IntModP import IntModP
from src.CurvePoint import CurvePoint
from src.EllipticCurve import EllipticCurve


from src.Emitter import Emitter
from src.Receiver import Receiver
from src.Interceptor import Interceptor


from src.utils import initPublic, displayPointsArray

In [2]:
IntModP.p = 46337
a = 121
b = 42

In [3]:
E = EllipticCurve(a, b)

In [4]:
print(len(E.points))
# Q = E.getGeneratorPoint(23374)
Q = E.getPointOfOrder(46000)
print(Q)

0
Point de E(Z/46337Z) : (7 [46337], 11520 [46337])


In [5]:
print(Q.order())

46184


In [6]:
# Q = CurvePoint(a, b, 2, 2)
print(Q.order())
public = initPublic(Q)

46184


In [15]:
# A = Emitter(f"Message avec p = {IntModP.p} !!", public)
A = Emitter("MESSAAAAAGE avec des accents éèàôî$$€€£", public)
B = Receiver(public)

In [16]:
A.encodeMsg(public)

In [17]:
B.decodeMsg(public)

'MESSAAAAAGE avec des accents éèàôî$$€€£'

In [10]:
arr = [(el.x, el.y) for el in public["bP"][1]]

In [11]:
# p     , a  , b  --> #E(Fp) --> (xp, yp   , ordre )   (tps pour récupérer tous les points de E, tps pour trouver un élément d'ordre q)
# 7879  , 7  , 8  --> 7944   --> (13, 2232 , 3972  )   (1m46 / 1s)  --> 25s
# 10007 , 121, 42 --> 10047  --> (2 , 819  , 10047 )   (2m56  / 1s) --> 40.7s
# 15013 , 121, 42 --> 14968  --> (0 , 5394 , 7484  )   (6m43  / 1s)
# 20011 , 121, 42 --> 20016  --> (13, 2691 , 20016 )   (11m52 / 3s)
# 40009 , 121, 42 --> 40354  --> (9 , 10554, 40354 )   (45m35 / 8s)
# 100003, 121, 42 --> 100413 --> (1 , 7583 , 100413)   (307m  / 5s)


### Génération de courbes et temps d'exécutions
|    $p$   	| $a$ 	| $b$ 	| $\#E(F_p)$ 	|  $(x_P, y_P)$ 	|  $P.o$  	| `EllipticCurve(a, b)` |     `getGeneratorPoint()` 	|
|:--------:	|:---:	|:---:	|:----------:	|:-------------:	|:------:	|:-------------------:	|:-------------------------:	|
|  $7879$  	|  7  	|  8  	|    7944    	|  $(13, 2232)$ 	|  3972  	|        25s        	|            1.3s            	|
|  $10007$ 	| 121 	|  42 	|    10047   	|  $(2 , 819)$  	|  10047 	|        39.6s        	|            0.5s              	|
|  $15013$ 	| 121 	|  42 	|    14968   	|  $(0 , 5394)$ 	|  7484  	|        1m28          	|            0.5s              	|
|  $20011$ 	| 121 	|  42 	|    20016   	|  $(13, 2691)$ 	|  20016 	|        2m41          	|            2.9s             	|
|  $40009$ 	| 121 	|  42 	|    40354   	|  $(9 , 10554)$ 	|  40354 	|        11m8          	|            8.1s             	|
|$100\ 003$	| 121 	|  42 	|   100413   	|  $(1 , 7583)$ 	|  100413 	|        70m           	|            4.1s             	|